# Sound Object Playback

The purpose of this script is to break out some 5-second sound objects from the Metatone touch archive and play them back on a device.

In [107]:
#%matplotlib inline
from __future__ import print_function
import os
import time
from datetime import timedelta
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
import seaborn as sns
import random
#from touch_performance_player import TouchPerformancePlayer
import OSC
import sys
sys.path.append("./MetatoneClassifier/classifier/")
import metatone_classifier
import generate_posthoc_gesture_score
import pickle

def clean_sound_object(frame):
    """Cleans up sound object frames by removing unneeded 
        columns and changing times to differences."""
    first_time = frame.index[0].to_pydatetime()
    output = frame[['x_pos','y_pos','velocity']]
    output['time'] = output.index
    output.time = (output.time - first_time).apply(timedelta.total_seconds)
    output.time = output.time.fillna(0)
    output = output.rename(columns={'x_pos': 'x', 'y_pos': 'y', 'velocity': 'velocity', 'time': 'time'})
    return output

def five_second_frame(row, frame):
    """
    Returns a dataframe of five seconds length before "time".
    """
    end_time = row.name
    frame = frame.between_time((end_time + timedelta(seconds=-5)).time(), end_time.time())
    if frame.empty:
        return np.nan
    return frame

def add_gestures_to_objects_series(objects_series):
    """Classifies gestures from a series of sound-object frames."""
    feature_vectors = objects_series.apply(generate_posthoc_gesture_score.feature_vector_from_frame).apply(pd.Series)
    out = pd.DataFrame({'objects':objects_series})
    feature_vectors = feature_vectors[metatone_classifier.FEATURE_VECTOR_COLUMNS]
    preds = generate_posthoc_gesture_score.CLASSIFIER.classifier.predict(feature_vectors)
    out['gestures'] = preds
    return out

def convert_log_frame_to_classified_sound_objects(log):
    """Converts a touch-log into a dataframe of sound-objects and classified gestures."""
    sound_object_frames = []
    for n in log.device_id.unique():
        log_frame = log[log.device_id == n]
        times = pd.DataFrame(log_frame.index,index=log_frame.index).resample('1s').count()
        sound_objects = times.apply(five_second_frame, axis=1, frame=log_frame).dropna()
        sound_objects = add_gestures_to_objects_series(sound_objects)
        print("Classified",len(sound_objects.index),"sound objects.")
        sound_object_frames.append(sound_objects)
    out = pd.concat(sound_object_frames)
    print("Converting touch logs to sound_object format")
    out.objects = out.objects.apply(clean_sound_object)
    print("Total sound objects classified is:",len(out.index))
    return out

### Load up data

- Loads metatone logs.
- Checks if output directory exists, otherwise creates it.

In [2]:
log_files = []
performances = []

output_directory = "individual-perfs-metatone-format/"
output_fileending = "-indperf.csv"

if not os.path.exists(output_directory):
    print("Creating Individual Perf Directory")
    os.makedirs(output_directory)

for local_file in os.listdir("data"):
    if local_file.endswith("-touches.csv"):
        log_files.append("data/" + local_file)
# for log in log_files:
    # do some stuff
    
log_frames = []
print("Loading all the frames.")
for log in log_files:
    log_frames.append(pd.DataFrame.from_csv(log,parse_dates=True,header=0))
print("Done Loading", len(log_frames), "logs.")

Loading all the frames.
Done Loading 163 logs.


In [103]:
"""Class for playing back sound objects on metatone apps."""
import OSC
import random
import pandas as pd
from threading import Timer

PLAYBACK_TOUCH_PATTERN = "/metatone/playback/touch"
PLAYBACK_GESTURE_PATTERN = "/metatone/playback/gesture"

class TouchPerformancePlayer:
    """Sends sound objects to an iPad for concatenative performance."""

    def __init__(self, name, address, port):
        self.client = OSC.OSCClient()
        self.performers = {}
        self.name = name
        self.address = (address,port)
        self.addPerformer(name,address,port)
        self.current_gesture = 0
        self.sound_objects = pd.DataFrame()
        self.timers = []
        self.empty_object = pd.DataFrame(columns = ["x","y","velocity","time"])
        self.available_gestures = [0]
        self.playing = False;
        print("Initialising a performance player for", name, "at", address + ":" + str(port))
        
    def setPerformances(self,sound_objects):
        """Set a dataframe of soundobjects for performances. DataFrame should have two column: objects and gestures"""
        self.sound_objects = sound_objects
        print("Setting up new sound_object dataframe:")
        print(self.sound_objects.gestures.value_counts())
        self.available_gestures = self.sound_objects.gestures.unique().tolist()
        self.available_gestures.append(0)
        print("Available Gestures:", self.available_gestures)

    def addPerformer(self, name, address, port):
        """Adds a performer's address and port to the list"""
        self.performers[name] = (address, port)
        
    def startPlaying(self):
        """Start sending sound objects to the performer."""
        if not self.playing:
            print("Starting Playback with gesture:", self.current_gesture)
            self.continuePlaying();
            
    def stopPlaying(self):
        """Stop sending sound objects to the performer and stop all timers."""
        if self.playing:
            print("Stopping playback and cancelling timers for:", self.name)
            self.cancelTimers()
            self.playing = False;
            self.current_gesture = 0;
        
    def continuePlaying(self):
        """Continue playing the gesture previously set."""
        print("Playing perf-object with gesture:", self.current_gesture, "on:", self.name)
        self.scheduleSoundObject(self.current_gesture)
        
    def updateGesture(self,new_gesture):
        """Update the gesture and start playing if it's different from before."""
        if self.current_gesture != new_gesture:
            # start playing new gesture immediately.
            self.current_gesture = new_gesture
            self.scheduleSoundObject(self.current_gesture)
    
    def cancelTimers(self):
        """Stop all current timers to send new performance objects."""
        for t in self.timers:
            t.cancel()
        self.timers = []
    
    def sendTouch(self, performer, x, y, velocity):
        """Sends an OSC message to trigger a touch sound."""
        address = self.performers[performer]
        self.client.sendto(OSC.OSCMessage(PLAYBACK_TOUCH_PATTERN, [performer,x, y, velocity]), address)
        
    def scheduleSoundObject(self,gesture):
        """Schedule a random sound object from a given gesture."""
        if gesture not in self.available_gestures:
            print("Sound object for gesture",gesture,"is not available.")
            return
        if gesture == 0:
            perf = self.empty_object
        else:
            perf = self.sound_objects[self.sound_objects.gestures == gesture].objects.sample(1)[0]
        self.schedulePerformance(perf,self.name)
        t = Timer(5.0,self.continuePlaying)
        self.timers.append(t)
        t.start()

    def schedulePerformance(self, perf, performer = "local"):
        """Schedule performance of a tiny performance dataframe."""
        self.cancelTimers()
        self.playing = True;
        if (perf.empty): # Returns if the perf is empty.
            return
        for row in perf.iterrows():
            self.timers.append(Timer(row[1]['time'],self.sendTouch,args=[performer,row[1]['x'],row[1]['y'],row[1]['velocity']]))
        for t in self.timers:    
            t.start()

## Loading gesture to sound-object data

In [111]:
# Load or generate the sound_object to classified object frame and save as a pickle.
gesture_to_object_filename = "./gesture_to_sound_object_dataframe.pickle"
try:
    pickle_file = open(gesture_to_object_filename, "rb")
    cla_frame = pickle.load(pickle_file)
    pickle_file.close()
    print("### Classified sound objects successfully loaded.  ###")
except:
    print("### Error loading sound objects, classifying again.")
    cla_frame = convert_log_frame_to_classified_sound_objects(log_frames[120])
    with open(gesture_to_object_filename, 'wb') as f:
        pickle.dump(cla_frame, f, pickle.HIGHEST_PROTOCOL)
    print("### Classified sound objects successfully generated and saved.  ###")

### Classified sound objects successfully loaded.  ###


## Test out the sound_object playback

In [105]:
## When we get home, make this work.
## Need to find out address of iPads manually using bonjour browser.

player = TouchPerformancePlayer('charles','192.168.0.36',51200)
player.setPerformances(cla_frame)
player.startPlaying()

Initialising a performance player for charles at 192.168.0.36:51200
Setting up new sound_object dataframe:
2    591
4    385
3    381
7    223
8    209
1    188
6     29
5     21
Name: gestures, dtype: int64
Available Gestures: [8, 7, 4, 3, 1, 2, 5, 6, 0]
Starting Playback with gesture: 0
Playing perf-object with gesture: 0 on: charles
Playing perf-object with gesture: 0 on: charles


In [109]:
player.updateGesture(12)
player.stopPlaying()

Stopping playback and cancelling timers for: charles
